# Scaling from mendes santos paper
1. **import data** 
1. **define scaling functions**
1. **make parameter mesh**
1. **plot 3d scatter**

In [71]:
import numpy as np
import matplotlib.pyplot as plt
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from fssa import quality
from tqdm import tqdm
import plotly.graph_objects as go
import numpy as np
import seaborn as sns

## Import data

In [106]:
rho = np.linspace(0.1,6.1,31)
Ls= np.arange(8,16,2).astype(float)

Y = np.zeros((len(Ls),len(rho)))
dY = np.zeros((len(Ls),len(rho)))
for index, L in enumerate(Ls):
    data = np.load('data/cluster/'+'2nn-{}-combined.npy'.format(int(L)))
    Y[index] = np.mean(data,axis=0)
    dY[index] = np.std(data,axis=0)
    

rho = rho[5:]
Y = Y[:,5:]
dY = dY[:,5:]

## scale data func.

In [107]:
def x_dat_2_new(rho, rho_c, l, nu):
    return np.power(l, 1./nu) * (rho-rho_c)

def y_dat_new(y, l, zeta,nu):
    return np.power(l, -zeta/nu) * y


l_mesh_local, rho_mesh_local = np.meshgrid(Ls, rho, indexing='ij')

def goal_function(rho_c = 3., nu=2.2, zeta=1.2):

    XX = x_dat_2_new(rho=rho_mesh_local, rho_c=rho_c, l=l_mesh_local ,nu=nu) 
    YY = y_dat_new(y=Y, l=l_mesh_local, zeta=zeta, nu=nu)
    dYY = y_dat_new(y=dY, l=l_mesh_local, zeta=zeta, nu=nu)
    return quality(XX,YY,dYY)
    

## Free param mesh

In [111]:
rho_c_for_mesh = np.linspace(3.4,4.,30)
nu_for_mesh = np.linspace(1,2.5,5)
zeta_for_mesh = np.linspace(1,2.5,5)

rho_c_mesh, nu_mesh, zeta_mesh = np.meshgrid(rho_c_for_mesh,nu_for_mesh,zeta_for_mesh,indexing='ij')
quality_mesh = np.zeros(np.shape(rho_c_mesh))

## calculate quality for each mesh point

In [112]:
for index3, rho_3, nu_3, z_3 in tqdm(zip(range(len(rho_c_mesh)),rho_c_mesh , nu_mesh, zeta_mesh)):
    #print(rho_3, nu_3, a_3,'\n')
    for index2, rho_2, nu_2, z_2 in zip(range(len(rho_3)),rho_3, nu_3, z_3):
        #print(rho_2, nu_2, a_2,'\n')
        for index1, rho_1, nu_1, z_1 in zip(range(len(rho_2)),rho_2, nu_2, z_2):
            quality_mesh[index3,index2,index1] = 1/goal_function(rho_1, nu_1, z_1)
            #print(quality_mesh)

30it [00:06,  4.96it/s]


## Plot

In [113]:
### import plotly.graph_objects as go
import numpy as np

size = 1/((quality_mesh.flatten()/np.max(quality_mesh.flatten()))**1)
size = (quality_mesh.flatten()/np.max(quality_mesh.flatten()))**4*30

fig = go.Figure(data=[go.Scatter3d(x=rho_c_mesh.flatten(), 
                                   y=nu_mesh.flatten(), 
                                   z=zeta_mesh.flatten(),
                                   mode='markers',
                                  marker=dict(
                                            size=size,
                                            color=size,          # set color to an array/list of desired values
                                    colorscale='inferno',   # choose a colorscale
                                    opacity=0.8,
                                      
                                ))])
fig.show()

In [ ]:
i had used the scaling func from Mendez-santos: where $x = np.power(l, -a) * y$ and $y = np.power(l, 1./nu) * (rho-rho_c)$. Now having changed to $x = np.power(l, -zeta/nu) * y$